In [5]:
JETSON_INTERFACE_PATH = '../../../jetson-inference'
JETSON_INTERFACE_SSD_PATH = f'{JETSON_INTERFACE_PATH}/python/training/detection/ssd'
DATA_DIR='/nvdli-nano/data/objectdetection/images/custom/'
MODEL_DIR='/nvdli-nano/data/objectdetection/model/custom'
TEST_DIR=f'{DATA_DIR}/test'
DEFAULT_PRETRAINED_MODEL=f'{JETSON_INTERFACE_SSD_PATH}/models/mobilenet-v1-ssd-mp-0_675.pth'
DETECTED_DIR = f'{TEST_DIR}/detected' 
import os
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    os.makedirs(DETECTED_DIR)

In [6]:
import sys
sys.path.append(os.path.abspath(JETSON_INTERFACE_SSD_PATH))
sys.path.append(os.path.abspath(f'{JETSON_INTERFACE_PATH}/build/aarch64/bin'))

In [7]:
NUM_EPOCHS = 10
BATCH = 4
LR = 0.01

In [4]:
if not os.path.exists(DEFAULT_PRETRAINED_MODEL):
    os.system(f"wget --quiet --show-progress --progress=bar:force:noscroll --no-check-certificate https://nvidia.box.com/shared/static/djf5w54rjvpqocsiztzaandq1m3avr7c.pth -O {DEFAULT_PRETRAINED_MODEL}")

In [15]:
!python3 {JETSON_INTERFACE_SSD_PATH}/train_ssd.py --dataset-type=voc --data={DATA_DIR} --model-dir={MODEL_DIR} --batch-size={BATCH} --epochs={NUM_EPOCHS} --lr={LR} --pretrained-ssd={DEFAULT_PRETRAINED_MODEL}

2024-02-19 12:55:35 - Using CUDA...
2024-02-19 12:55:35 - Namespace(balance_data=False, base_net=None, base_net_lr=0.001, batch_size=4, checkpoint_folder='/nvdli-nano/data/objectdetection/model/custom', dataset_type='voc', datasets=['/nvdli-nano/data/objectdetection/images/custom/'], debug_steps=10, extra_layers_lr=None, freeze_base_net=False, freeze_net=False, gamma=0.1, lr=0.01, mb2_width_mult=1.0, milestones='80,100', momentum=0.9, net='mb1-ssd', num_epochs=10, num_workers=2, pretrained_ssd='../../../jetson-inference/python/training/detection/ssd/models/mobilenet-v1-ssd-mp-0_675.pth', resume=None, scheduler='cosine', t_max=100, use_cuda=True, validation_epochs=1, weight_decay=0.0005)
2024-02-19 12:55:35 - Prepare training datasets.
2024-02-19 12:55:35 - No labels file, using default VOC classes.
2024-02-19 12:55:35 - Stored labels into file /nvdli-nano/data/objectdetection/model/custom/labels.txt.
2024-02-19 12:55:35 - Train dataset size: 20
2024-02-19 12:55:35 - Prepare Validation 

In [5]:
!python3 {JETSON_INTERFACE_SSD_PATH}/onnx_export.py --model-dir={MODEL_DIR}

^C


In [8]:
import threading
import time
import torch.nn.functional as F
import jetson.inference
import jetson.utils
import ipywidgets
from jetcam.utils import bgr8_to_jpeg
# USB Camera (Logitech C270 webcam)
camera = jetson.utils.videoSource("/dev/video0") 
net = jetson.inference.detectNet(argv=[f'--model={MODEL_DIR}/ssd-mobilenet.onnx', 
                                       f'--labels={MODEL_DIR}/labels.txt', 
                                       '--input-blob=input_0', 
                                       '--output-cvg=scores', 
                                       '--output-bbox=boxes'], threshold=0.90)

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
with open("./utils/ready.jpeg", "rb") as file:
    default_image = file.read()
prediction_widget = ipywidgets.Image(format='jpeg', width=300, height=300, value=default_image)

def live(state_widget, net, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.Capture()
        if image is None: # capture timeout
            continue
        detections = net.Detect(image, overlay='box,labels,conf')
        bgr_img = jetson.utils.cudaAllocMapped(width=image.width,
                          height=image.height, format='bgr8')
        jetson.utils.cudaConvertColor(image, bgr_img)
        prediction_widget.value = bgr8_to_jpeg(jetson.utils.cudaToNumpy(bgr_img))
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, net, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

# display(live_execution_widget)
print("live_execution_widget created")
from IPython.display import display
display(live_execution_widget)

live_execution_widget created
